In [ ]:
import pandas as pd
import numpy as np
import os
import sys

# Load other functions
sys.path.append(os.path.abspath('../src'))
sys.path.append(os.path.abspath('../scripts'))

from data_loader import CSVDataloader, get_file_path

In [ ]:
# Call functions
data = CSVDataloader()

## Load Credit Risk data for Analysis

In [ ]:
# Load AAPL data
data = get_file_path("AAPL")
AAPL_data = CSVDataloader(data).load_data()

In [ ]:
# Call functions
data = CSVDataloader()